In [160]:
import findspark
findspark.init('C:\Spark\spark-3.0.1-bin-hadoop2.7')

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PD") \
    .getOrCreate()

path = "C:\\Users\\Leticia Pereira\\Documents\\Datasets - Teste Data Engineer - Passei Direto\\" #substituir

courses = spark.read.json(path+"BASE A\\courses.json").createOrReplaceTempView("cursos")
sessions = spark.read.json(path+"BASE A\\sessions.json").createOrReplaceTempView("sessoes")
stud_follow = spark.read.json(path+"BASE A\\student_follow_subject.json").createOrReplaceTempView("seguidas")
students = spark.read.json(path+"BASE A\\students.json").createOrReplaceTempView("alunos")
DFstudents = spark.read.json(path+"BASE A\\students.json")
subjects = spark.read.json(path+"BASE A\\subjects.json").createOrReplaceTempView("assuntos")
subscriptions = spark.read.json(path+"BASE A\\subscriptions.json").createOrReplaceTempView("assinaturas")
universities = spark.read.json(path+"BASE A\\universities.json").createOrReplaceTempView("universidades")
DFuniversities = spark.read.json(path+"BASE A\\universities.json")
pageViews = spark.read.json(path+"BASE B\\*json").createOrReplaceTempView("pageViews")

In [117]:
# pageViews.printSchema()

df = spark.sql("SELECT session_uuid, city_name, marketing_campaign, user_type, marketing_source, `Page Category`, SUBSTR(studentId_clientType, 0, INSTR(studentId_clientType, '@')-1) AS studentId FROM pageViews")
df.groupBy("Page Category").count().orderBy('count', ascending=False).show(truncate=False)

+---------------------+------+
|Page Category        |count |
+---------------------+------+
|disciplina           |335880|
|home                 |71624 |
|cadastro             |56984 |
|busca                |41595 |
|arquivo              |38527 |
|premium              |35924 |
|pergunta             |30006 |
|perfil               |26475 |
|lista                |12598 |
|listas               |12310 |
|pagamento            |9608  |
|configuracoes        |7642  |
|exercicios-resolvidos|5911  |
|explorar-materiais   |5188  |
|ranking              |2679  |
|meu-premium          |1151  |
|materiais-de-estudo  |985   |
|estude               |752   |
|topico               |722   |
|mensagens            |376   |
+---------------------+------+
only showing top 20 rows



In [158]:
df.createOrReplaceTempView("navegacao")
# studentPage = spark.sql("SELECT nav.*, alu.*, uni.Name FROM navegacao nav INNER JOIN alunos alu ON nav.student = alu.Id INNER JOIN universidades uni ON alu.UniversityId = uni.Id") 

ligacao = df.join(DFstudents, df.studentId == DFstudents.Id, how='inner').drop("Id")

ligacao = ligacao.join(DFuniversities, DFuniversities.Id == ligacao.UniversityId, how='inner')

ligacao.show()


+--------------------+---------+------------------+---------+----------------+-------------+--------------------+------+--------+--------------------+------------+-------+-------------+------------+------+-------+
|        session_uuid|city_name|marketing_campaign|user_type|marketing_source|Page Category|           studentId|  City|CourseId|      RegisteredDate|SignupSource|  State|StudentClient|UniversityId|    Id|   Name|
+--------------------+---------+------------------+---------+----------------+-------------+--------------------+------+--------+--------------------+------------+-------+-------------+------------+------+-------+
|7b20734920adf179c...|     null|              null|    known|            null|   disciplina|04f5634512e7a853f...|Maceió| 1199517|2017-10-02 13:42:...|      Google|Alagoas|      Website|      664623|664623|ESTÁCIO|
|7b20734920adf179c...|     null|              null|    known|            null|   disciplina|04f5634512e7a853f...|Maceió| 1199517|2017-10-02 13:4